In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies = pd.read_csv("Datasets/tmdb_5000_movies.csv")
credits = pd.read_csv("Datasets/tmdb_5000_credits.csv")

In [ ]:
movies.head(1)

In [ ]:
credits.head(1)

## Merge two dataframes

In [ ]:
movies = movies.merge(credits, on='title')

In [ ]:
movies.head(1)

## We will keep this columns only 
1. **genres**
2. **id**
3. **keywords**
4. **title**
5. **overview**
6. **cost**
6. **crew**

In [ ]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [ ]:
movies.head(1)

**In our final dataframe we will have only three columns - movie_id, title, tags**

According to this we will now prepare the data

In [ ]:
movies.isnull().sum() # check if there is any missing data

In [ ]:
movies.dropna(inplace=True) # remove missing values

In [ ]:
movies.duplicated().sum() # check for duplicate rows

In [ ]:
movies.iloc[0].genres 

In [ ]:
# '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
# ['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [ ]:
# def convert(obj):
#     l = []
#     for i in obj:
#         l.append(i['name'])
#         return l

# convert(movies.iloc[0].genres)

```
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[23], line 1
----> 1 convert(movies.iloc[0].genres)

Cell In[22], line 4
      2 l = []
      3 for i in obj:
----> 4     l.append(i['name'])
      5     return l

TypeError: string indices must be integers
```

- **this problem is expected because this is a string of lists**
- **to fix this we need to convert this into a list**
- **we will use ast python module for this.**
(```ast.literal_eval```)

In [ ]:
import ast
def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i["name"])
    return l

In [ ]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
movies.head()

In [ ]:

def convert2(obj):
    l = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            l.append(i["name"])
            counter += 1
        else:
            break
    return l

In [ ]:
movies['cast'] = movies['cast'].apply(convert2)

In [ ]:
movies.head()

In [ ]:
def fetch_director(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l.append(i["name"])
            break
    return l

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies.head()

In [ ]:
movies['overview'][0]

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x: x.split()) # convert overview text into a list of words

In [ ]:
movies.head()

## Now we need to remove spaces between the names to convert them into a single word

- **Science Fiction -> ScienceFiction**
- **Johnny Depp -> JohnnyDepp**

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ", "") for i in x]) # convert genres text into a list of genres
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ", "") for i in x]) # convert genres text into a list of genres
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ", "") for i in x]) # convert genres text into a list of genres
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ", "") for i in x]) # convert genres text into a list of genres

In [ ]:
movies.head()

## tags = overview + genres + keywords + cast + crew 

concatinate

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies.head()

In [ ]:
final_df = movies[['movie_id', 'title', 'tags']]

In [ ]:
final_df

In [ ]:
final_df['tags'] = final_df['tags'].apply(lambda x: " ".join(x)) # convert list of words into a single string

In [ ]:
final_df.head()

In [ ]:
# convert to lower case
final_df['tags'] = final_df['tags'].apply(lambda x: x.lower())

In [ ]:
final_df.head()

## apply stemming 

**['love', 'loving', 'loved']** <br/>
**['love', 'love', 'love']**

In [ ]:
import nltk

In [ ]:
from nltk.stem import PorterStemmer

ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []

    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
final_df['tags'] = final_df['tags'].apply(stem) # apply stemming to the tags

# Using Scikit Learn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(final_df['tags']).toarray() # convert tags into a matrix of token counts

In [ ]:
cv.get_feature_names_out() # get the feature names (words) from the CountVectorizer

In [ ]:
"""
['love', 'loving', 'loved']
-- apply stemming (cell no. -> 37)
['love', 'love', 'love']
"""

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors) # calculate the cosine similarity between the vectors

In [ ]:
similarity.shape # check the shape of the similarity matrix

In [ ]:
similarity[0] # check the similarity scores for the particular movie

In [ ]:
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x: x[1])[1:6] # get the top 5 similar movies

In [ ]:
def recomended(movie):
    movie_idx = final_df[final_df['title'] == movie].index[0] # get the index of the movie
    distances = similarity[movie_idx] # get the similarity scores for the movie
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    for i in movies_list:
        print(final_df.iloc[i[0]]['title'])

In [ ]:
recomended('Batman') # test the recommender system with a movie title

In [ ]:
import pickle

In [ ]:
pickle.dump(final_df, open('movies.pkl', 'wb')) # save the final dataframe to a pickle file